In [3]:
# First step: Difference of gaussian to enhance centriole signal
import cv2
import numpy as np
from PIL import Image

#from numba import jit

from time import asctime

# For Otsu
from skimage.morphology import disk, remove_small_objects
from skimage.filters import threshold_otsu, rank

from scipy import ndimage

In [4]:
# PERFORM THE DIFFERENCE BETWEEN TWO GAUSSIAN BLUR
def difference_of_gaussian(img, sigma_1 = 0.5, sigma_2 = 5, kernel_ = 51):
    """ 
    Function that extract centriole from a preprocess image using Difference-of-Gaussian and Otsu tresholding
    Parameters:
    img = the image on which you want to extract the centriole
    sigma_1 = the 'small' sigma for gaussian blur (0.5 by default)
    sigma_2 = the 'high' sigma for gaussian blur (5 by default)
    kernel_ = the size of the kernel to perform gaussian blur (5 by default)
    """
    
    # Gaussian Blur
    sigma1 = cv2.GaussianBlur(img, (kernel_, kernel_), sigma_1)
    sigma2 = cv2.GaussianBlur(img, (kernel_, kernel_), sigma_2)

    # Gaussian of differences
    dog = np.where(sigma1>sigma2, sigma1-sigma2, 0)
    
    # 8bits transformation
    dog = map_uint16_to_uint8(dog)      

    return dog



# REMOVE OBJECT SMALLER THAT x PIXELs
def small_object_remover(image, obj_size = 0):
    image = np.array(image , bool)
    image = remove_small_objects(image, obj_size)
    image = image.astype(int)
    
    return image
    
    
    
# REMOVE OBJECT BIGGER THAT x Pixels
def big_object_remover(image, obj_size = 1000):
    img = image - small_object_remover(image, obj_size)
    
    return img


# REMOVE OBJECT SMALLER THAT x AND BIGGER THAT y
def object_remove_treshold(image, mini, maxi):
    rm_small = small_object_remover(image, mini)
    rm_big = big_object_remover(rm_small, maxi)
    
    return rm_big


# TRANSFORM A 16 BYTES IMAGES INTO 8 BYTES USING A MIN/MAX NORMALIZATION
def map_uint16_to_uint8(img, lower_bound=None, upper_bound=None):
    '''
    Map a 16-bit image trough a lookup table to convert it to 8-bit.

    Parameters
    ----------
    img: numpy.ndarray[np.uint16]
        image that should be mapped
    lower_bound: int, optional
        lower bound of the range that should be mapped to ``[0, 255]``,
        value must be in the range ``[0, 65535]`` and smaller than `upper_bound`
        (defaults to ``numpy.min(img)``)
    upper_bound: int, optional
       upper bound of the range that should be mapped to ``[0, 255]``,
       value must be in the range ``[0, 65535]`` and larger than `lower_bound`
       (defaults to ``numpy.max(img)``)

    Returns
    -------
    numpy.ndarray[uint8]
    '''
    if lower_bound is None:
        lower_bound = np.min(img)
        
    if upper_bound is None:
        upper_bound = np.max(img)
        
    if not(0 <= lower_bound < 2**16) and lower_bound is not None:
        raise ValueError(
            '"lower_bound" must be in the range [0, 65535]')
        
    if not(0 <= upper_bound < 2**16) and upper_bound is not None:
        raise ValueError(
            '"upper_bound" must be in the range [0, 65535]')

    if lower_bound >= upper_bound:
        raise ValueError(
            '"lower_bound" must be smaller than "upper_bound"')
        
    lut = np.concatenate([
        np.zeros(lower_bound, dtype=np.uint16),
        np.linspace(0, 255, upper_bound - lower_bound).astype(np.uint16),
        np.ones(2**16 - upper_bound, dtype=np.uint16) * 255
    ])
    
    return lut[img].astype(np.uint8)



# PERFORM A LOCAL TRESHOLDING USING OTSU METHOD

def otsu_local_treshold(img, kernel_radius = 10):
    radius = kernel_radius
    selem = disk(radius)

    local_otsu = rank.otsu(img, selem)

    return img >= local_otsu

In [1]:

def dog_and_otsu(img, otsu_kernel=1000, min_obj_size=10, max_obj_size=200):
    # Trnasform image into uint8
    img = map_uint16_to_uint8(img)

    # Performer differential of gaussian
    dog = difference_of_gaussian(img)
    print(f'{asctime()}: Differential of Gaussian performed')
    
    # Threshold the image
    otsu = otsu_local_treshold(dog, otsu_kernel)
    print(f'{asctime()}: Thresholding performed')

    # Treshold object size
    img = object_remove_treshold(otsu, min_obj_size, max_obj_size)
    print(f' {asctime()}: Object size thresholded')

    return img

In [6]:
#@jit(nopython=True)
def isWithin(x, y, direction, width, height):
    #Depending on where we are and where we are heading, return the appropriate inequality.
    xmax = width - 1;
    ymax = height -1;
    if direction ==0:
        return (y>0);
    elif direction ==1:
            return (x<xmax and y>0);
    elif direction ==2:
            return (x<xmax);
    elif direction ==3:
            return (x<xmax and y<ymax);
    elif direction ==4:
            return (y<ymax);
    elif direction ==5:
            return (x>0 and y<ymax);
    elif direction ==6:
            return (x>0);
    elif direction ==7:
            return (x>0 and y>0);

    return False;

#@jit(nopython=True)
def find_local_maxima_np(img_data):
    #This is the numpy/scipy version of the above function (find local maxima).
    #Its a bit faster, and more compact code.
    
    #Filter data with maximum filter to find maximum filter response in each neighbourhood
    max_out = ndimage.filters.maximum_filter(img_data,size=10)
    #Find local maxima.
    local_max = np.zeros((img_data.shape))
    local_max[max_out == img_data] = 1
    local_max[img_data == np.min(img_data)] = 0
    
    return local_max

In [8]:
#@jit(nopython=True)
def find_maxima(img, noise_tolerance = 10):
    ntol = noise_tolerance #Noise Tolerance.

    #Start of script
    img_data = np.array(img)

    #Should your image be an RGB image.
    if len(img_data.shape) > 2:
        img_data = (np.sum(img_data,2)/3.0)

    if np.max(img_data) > 255 or np.min(img_data) < 0:
        print('warning: your image should be scaled between 0 and 255 (8-bit).')

    #Finds the local maxima using maximum filter.
    local_max = find_local_maxima_np(img_data)

    #Find local maxima coordinates
    ypts, xpts = np.where(local_max == 1)
    #Find the corresponding intensities
    ipts = img_data[ypts,xpts]

    #Changes order from max to min.
    ind_pts = np.argsort(ipts)[::-1]
    ypts = ypts[ind_pts]
    xpts = xpts[ind_pts]
    ipts = ipts[ind_pts]


    #Create our variables and allocate memory for speed.
    types = np.array(local_max).astype(np.int8)
    pListx = np.zeros((img_data.shape[0]*img_data.shape[1]))
    pListy = np.zeros((img_data.shape[0]*img_data.shape[1]))
    width = img_data.shape[1]
    height = img_data.shape[0]

    #This defines the pixel neighbourhood 8-connected neighbourhood [3x3]
    dir_x = [0,  1,  1,  1,  0, -1, -1, -1]
    dir_y = [-1, -1,  0,  1,  1,  1,  0, -1]

    """ 
    FAIRE UN NEIGHBOURHOUD OF SIZE X automatically

    """

    #At each stage we classify our pixels. We use 2n as we can use more than one definition
    #together.
    MAXIMUM = 1
    LISTED = 2
    PROCESSED = 4
    MAX_AREA = 8
    EQUAL = 16
    MAX_POINT = 32
    ELIMINATED = 64

    maxSortingError = 0
    #Now we iterate through each of a local maxima and prune the sub-maximal peaks.
    #This extends the neighbourhood and combines and prunes away unwanted maxima using the
    #noise tolerance to decide what counts and what doesn't
    for y0, x0, v0 in zip(ypts, xpts, ipts):

        if (types[y0,x0]&PROCESSED) !=0:
            #If processed already then skip this pixel, it won't be maxima.
            continue


        sortingError = True
        while sortingError == True:

            #Our initial pixel 
            pListx[0] = x0
            pListy[0] = y0
            types[y0,x0] |= (EQUAL|LISTED) #Listed and Equal


            listlen = 1
            listI = 0

            #isEdgeMAxima = (x0==0 or x0 == width-1 or y0 == 0 or y0 == height -1)
            sortingError = False
            maxPossible = True
            xEqual = float(x0)
            yEqual = float(y0)
            nEqual = 1.0

            while listI < listlen:
                #We iteratively add points. This loop will keep going until we have
                #exhausted the neighbourhood.

                #Collect the next point to consider
                x = pListx[listI]
                y = pListy[listI]

                #Is our point legal. //not necessary, but faster than isWithin.
                #With subsequent 'OR' statement the first arguement is evaluated
                #and then only the second if the first is false.
                isInner = (y != 0 and y != height -1) and (x!=0 and x != width-1)

            
                for d in range(0,8):
                    #Scan the neighbourhood.
                    x2 = int(x+dir_x[d])
                    y2 = int(y+dir_y[d])


                    if (isInner or isWithin(x,y,d,width,height)) and (types[y2,x2]&LISTED) ==0:
                        #If the pixel is located legally


                        if types[y2,x2]&PROCESSED !=0:
                            #If the pixel is processed already. It won't be maxima.
                            maxPossible = False
                            break;

                        v2 = img_data[y2,x2] #return pixel from neighbourhood.

                        if v2 > v0 + maxSortingError:
                            #We have reached a higher maximum.
                            maxPossible = False
                            break;

                        elif v2 >= v0 - ntol:

                            #If equal or within we add it on.
                            pListx[listlen] = x2
                            pListy[listlen] = y2
                            listlen = listlen+1
                            #We mark it as listed. Because its in our list :-).
                            types[y2,x2] |= LISTED


                            #We are not excluding edge pixels yet.
                            #if (x2==0 or x2 == width-1 or y2==0 or y2==height-1):
                            #    isEdgeMaximum = True

                                #maxPossible = False
                                #break

                            if v2==v0:

                                #This point is equal to our maxima.
                                types[y2,x2] |= EQUAL
                                #We have to merge the coordinates.
                                xEqual += x2
                                yEqual += y2
                                nEqual += 1
                listI +=1
             
            #if sortingError:
                #If our point x0, y0 was not true maxima and we reach a bigger one, start again.
                #for listI in range(0,Listlen):
            #   types[pListy[0:listlen],pListx[0:listlen]] =0
            #else:
            if maxPossible == True:
                resetMask = ~(LISTED)
            else:
                resetMask = ~(LISTED|EQUAL)

            #Now we calculate the x and y-coordinates, if there were any equal.
            xEqual /= nEqual
            yEqual /= nEqual
            minDist2 = 1e20
            nearestI = 0

            #This makes sure it has same output as the fiji plugin. Not strictly needed.
            xEqual = round(xEqual)
            yEqual = round(yEqual)

            x = pListx[0:listlen].astype(np.int32)
            y = pListy[0:listlen].astype(np.int32)
            types[y,x] &= resetMask
            types[y,x] |= PROCESSED



            if maxPossible:
                types[y,x] |= MAX_AREA

                #This is where we assign the actual maxima location.
                dv =  (types[y,x]&EQUAL) !=0
                dist2 = (xEqual-x[dv]).astype(np.float64)**2+(yEqual-y[dv]).astype(np.float64)**2

                indx = np.arange(0,listlen)
                rd_indx = indx[dv]
                nearestI = rd_indx[np.argmin(dist2)]

                x = int(pListx[nearestI])
                y = int(pListy[nearestI])
                types[y,x] |= MAX_POINT


    out = types==61
    ypts,xpts = np.where(out)
    print(f"Centriole detected  {str(np.sum(out))}")
   
    
    # reformating coordinates
    coordinates = []
    for i in range(len(ypts)):
        coordinates.append((ypts, xpts))
    
    # Create a image with only centriole maxiam coordinates
    blank = np.zeros((img.shape[0], img.shape[1]))
                     
    for a_coord in coordinates:
        blank[a_coord] = 1
    """
    figsize(12,12)
    plot(xpts,ypts,'w+',markersize=10)
    imshow(img)
    """
                     
    return blank
